In [1]:
import os
print('Number of CPUs in the system: {}'.format(os.cpu_count()))

Number of CPUs in the system: 16


In [2]:
import sys
!{sys.executable} -m pip install ipyparallel

     |████████████████████████████████| 298 kB 11.6 MB/s eta 0:00:01


In [3]:
#import ipyparallel as ipp
#cluster = ipp.Cluster()
#cluster.start_cluster_sync()
#rc = cluster.connect_client_sync()

In [3]:
import sys
!{sys.executable} -m pip install tsfresh

     |████████████████████████████████| 97 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 21.0 MB/s eta 0:00:01 21.0 MB/s eta 0:00:01
     |████████████████████████████████| 9.9 MB 15.6 MB/s eta 0:00:01                         | 808 kB 15.6 MB/s eta 0:00:01 |███████▍                        | 2.3 MB 15.6 MB/s eta 0:00:01    | 7.3 MB 15.6 MB/s eta 0:00:016 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 16.6 MB/s eta 0:00:01�                         | 706 kB 16.6 MB/s eta 0:00:01��█████████             | 2.1 MB 16.6 MB/s eta 0:00:01��█████████████████████▏| 3.4 MB 16.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 25.6 MB/s eta 0:00:01�▌                         | 256 kB 25.6 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 34.6 MB 63 kB/s s eta 0:00:01   |▎                               |

  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.13.0
    Uninstalling protobuf-3.13.0:
      Successfully uninstalled protobuf-3.13.0
  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.0
    Uninstalling statsmodels-0.12.0:
      Successfully uninstalled statsmodels-0.12.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you 

In [4]:
import os
import ipyparallel as ipp
ipp.__version__   # check that at least version 7
mycluster = ipp.Cluster(n = int(os.cpu_count()))
mycluster.start_cluster_sync()

rc = mycluster.connect_client_sync()
rc.wait_for_engines(n = int(os.cpu_count()))
rc.ids


Using existing profile dir: '/home/idies/.ipython/profile_default'
Starting 16 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [7]:
dview = rc[:]
# Cause execution on main process to wait while tasks sent to workers finish
dview.block = True   
dview.apply(lambda : "Hello, World")


['Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World']

In [5]:
lview = rc.load_balanced_view() # default load-balanced view

In [10]:
#lview = c[:]
# Cause execution on main process to wait while tasks sent to workers finish
dview.block = True   
dview.apply(lambda : "Hello, World")


['Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World',
 'Hello, World']

In [8]:
dview.execute('from sklearn.ensemble import RandomForestRegressor as rfr')
dview.execute('import numpy as np')

<AsyncResult(execute): finished>

In [10]:
#dview.execute('pip install tsfresh')

In [9]:
dview.execute('from tsfresh import extract_features')
dview.execute('from tsfresh.utilities.dataframe_functions import impute')
dview.execute('from tsfresh.feature_extraction import ComprehensiveFCParameters')
dview.execute('import os')
dview.execute('import glob')
dview.execute('from astropy.table import Table')
dview.execute('import pandas as pd')

<AsyncResult(execute): finished>

In [15]:
def looFit(index, Ylocal, Xlocal):
    rf = rfr(n_estimators=100)
    fitted = rf.fit(np.delete(Xlocal, index, axis = 0), np.delete(Ylocal, index))
    pred = rf.predict(np.array([Xlocal[index, :]]))
    return(pred[0])


In [16]:
import numpy as np
X = np.random.random((200,5))
Y = np.random.random(200)
mydict = dict(X = X, Y = Y, looFit = looFit)
lview.push(mydict)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [17]:
# Need a wrapper function because map() only operates on one argument
def wrapper(i):
    return(looFit(i, Y, X))

n = len(Y)
import time
time.time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
pred = lview.map(wrapper, range(n))   # Run calculation in parallel
time.time()
print(pred[0:10])


[0.47273738651352487, 0.5758062069572971, 0.4575282041293287, 0.35179085505451463, 0.47816158567500927, 0.5290074019126499, 0.5368385671288748, 0.6138308849105077, 0.4704461210894488, 0.5287497557309716]


In [18]:
lview.execute('pip install tsfresh')

AttributeError: 'LoadBalancedView' object has no attribute 'execute'

In [11]:
%px a = np.random.random(4)

In [12]:
%px a.sum()

Out[0:2]: 2.0362446557326344

Out[2:2]: 1.8471508047363203

Out[7:2]: 1.6396417166100914

Out[5:2]: 1.778058458411706

Out[12:2]: 1.7256427838055366

...

Out[10:2]: 0.4350018523325364

Out[1:2]: 2.647963514660554

Out[15:2]: 1.478559525857312

Out[14:2]: 1.6551343272863592

Out[3:2]: 2.44962289944247

In [14]:
import numpy as np
a = np.random.random(4)

In [16]:
%px print(a)

[stdout:0] [0.22354866 0.80681176 0.96847607 0.03740816]


[stdout:1] [0.38070512 0.83089734 0.60472086 0.8316402 ]


[stdout:3] [0.50042788 0.60904967 0.69830824 0.6418371 ]


[stdout:2] [0.08200129 0.92907465 0.15192682 0.68414805]


[stdout:5] [0.16092716 0.09382458 0.77438712 0.74891961]


...

[stdout:9] [0.2616899  0.29438989 0.83074847 0.02188278]


[stdout:12] [0.05472815 0.65165895 0.3723624  0.64689329]


[stdout:11] [0.50738643 0.78621293 0.81967169 0.55832712]


[stdout:15] [0.27414393 0.44905122 0.45297488 0.30238948]


[stdout:13] [0.47451754 0.11712145 0.34772487 0.67490063]


In [17]:
%px os.chdir("/home/idies/workspace/Temporary/s.hossain18/scratch/data-extract/data")

In [18]:
%px extraction_settings = ComprehensiveFCParameters()

In [19]:
%px extension_1 = 'tbl'
%px import_filenames = [i for i in glob.glob('*.{}'.format(extension_1))]
%px df = pd.DataFrame()

In [25]:
%autopx
import_filenames = [i for i in glob.glob('*.{}'.format(extension_1))]
for f in import_filenames:
    read = Table.read(f, format='ascii.ipac')
    data = read.to_pandas()
    # over four threads on the local machine
    Distributor = MultiprocessingDistributor(n_workers=32,disable_progressbar=False,progressbar_title="Feature Extraction")

    ts_data = extract_features(data, column_id='set', column_sort='TIME', default_fc_parameters=extraction_settings,impute_function=impute,distributor=Distributor,chunksize=1)
    #df = df.append(ts_data)
    df = pd.concat([ts_data,df],ignore_index=True, sort=False)
    del ts_data

%autopx enabled


NameError: name 'glob' is not defined